# Script to prepare protein structures

- Protonates and charges proteins using Amber ff14SB.

In [1]:
# Imports
import contextlib
import os
import subprocess
import sys
from pathlib import Path

from tqdm import tqdm

import AutoDockTools
from meeko import MoleculePreparation, obutils
from openbabel import pybel
from parmed import load_file

In [7]:
def supress_stdout(func):
    def wrapper(*a, **ka):
        with open(os.devnull, 'w') as devnull:
            with contextlib.redirect_stdout(devnull):
                return func(*a, **ka)
    return wrapper

def remove_hetatoms_and_waters(pdb_in: str, pdb_out: str):
    with open(pdb_in, 'r') as f_in, open(pdb_out, 'w') as f_out:
        for line in f_in:
            if line.startswith("HETATM"):
                continue
            else:
                f_out.write(line)

class PrepProt(object): 
    def __init__(self, pdb_file): 
        self.prot = pdb_file
           
    def run_leap(self, pdb_file: str, output_prefix: str):
        leap_input = f"""
source leaprc.protein.ff14SB
protein = loadpdb "{pdb_file}"
check protein
saveamberparm protein {output_prefix}.prmtop {output_prefix}.inpcrd
savepdb protein {output_prefix}_ff14sb.pdb
quit
"""
        with open("leap.in", "w") as f:
            f.write(leap_input)
    
        result = subprocess.run(
            ["tleap", "-f", "leap.in"],
            stdout=subprocess.DEVNULL, stderr=subprocess.PIPE, text=True
        )
        if result.returncode != 0:
            print("LEaP failed:\n", result.stderr)
            raise RuntimeError("LEaP failed")

    def convert_to_pqr(self, prmtop: str, inpcrd: str, output_pqr: str):
        structure = load_file(prmtop, inpcrd)
        structure.save(output_pqr, format="pqr", overwrite=True)
        
    def addH(self, base, prot_pqr):
        self.prot_pqr = prot_pqr

        remove_hetatoms_and_waters(self.prot, f"{base}_nohetam.pdb")
        res=subprocess.run(["pdb4amber", "-i", f"{base}_nohetam.pdb", "-o", f"{base}_clean.pdb", "--dry", "--nohyd", "--reduce"],
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
                      )
        
        self.run_leap(f"{base}_clean.pdb", base)
        self.convert_to_pqr(f"{base}.prmtop", f"{base}.inpcrd", f"{base}.pqr")
    
    def get_pdbqt(self, base, prot_pdbqt):
        prepare_receptor = os.path.join(AutoDockTools.__path__[0], 'Utilities24/prepare_receptor4.py')
        subprocess.Popen(['python3', prepare_receptor, '-r', self.prot_pqr, '-o', prot_pdbqt, '-C', '-U', 'waters_nonstdres'],
                         stdout=subprocess.DEVNULL).communicate()#stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL).communicate()
        # Cleanup
        for fp in [f'{base}{suff}' for suff in ['_clean_nonprot.pdb',
                                                '_clean_renum.txt',
                                                '_clean_sslink',
                                                '_clean_water.pdb',
                                                '_clean.pdb',
                                                '_ff14sb.pdb',
                                                '_nohetam.pdb',
                                                '.inpcrd',
                                                '.pqr',
                                                '.prmtop',
                                               ]]:
            if os.path.exists(fp):
                os.remove(fp)
        for fp in ['leap.in',
                   'leap.log',
                   'parmed.in',
                   'reduce_info.log',
                  ]:
            if os.path.exists(fp):
                os.remove(fp)

In [3]:
# Load CASF-2016 baseline
pdb_ids, affs = [], []
with open('./data/CASF-2016/casf_2016.types', 'r') as f:
    for line in f:
        affs.append(line.strip().split()[1])
        pdb_ids.append(line.strip().split()[-1].split('/')[0])

In [4]:
# Prepare data
for pdb_id in tqdm(pdb_ids):
    protein_pqr = f'./data/CASF-2016/{pdb_id}/{pdb_id}_pocket.pqr'
    protein_pdbqt = f'./data/CASF-2016/{pdb_id}/{pdb_id}_pocket.pdbqt'

    prot = PrepProt(f'./data/CASF-2016/{pdb_id}/{pdb_id}_pocket.pdb')
    prot.addH(protein_pqr[:-4], protein_pqr)
    prot.get_pdbqt(protein_pqr[:-4], protein_pdbqt)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 285/285 [04:41<00:00,  1.01it/s]


In [5]:
# Write new types file
with open('./data/CASF-2016/casf_2016_prepared.types', 'w') as f:
    for pdb_id, aff in zip(pdb_ids, affs):
        f.write(f'1 {aff} {pdb_id}/{pdb_id}_pocket.pdbqt {pdb_id}/{pdb_id}_ligand.sdf\n')